## Single Party XGBoost on Data Subset
First we'll train an XGBoost model on a subset of the data. This simulates the federated setting in that a party will only have a subset of the data that's available to the aggregator for training. We'll look at the performance of a XGBoost model that's only trained on this subset. 
![title](img/exercise1.png)

Import the necessary libraries

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

Load in and examine the comma separated training data partition belonging to your party to get a better understanding of the data. 

Training data for the hospital dataset is at `/data/hospital/hospital_training_{party_id}.csv`. <br>
Training data for the insurance dataset is at `/data/insurance/insurance_training_{party_id}.csv`.

In [ ]:
# TODO: Read in the comma separated training data using the pandas.read_csv() function 
# and print out the first few rows of the dataset using the .head() function

training_data_path = # TODO
header = # TODO: set this to 0 if you're using the insurance dataset, None if you're using the hospital dataset

training_data = pd.read_csv(training_data_path, sep=",", header=header)
training_data.head()

In [ ]:
# Split the training dataset into features and labels
y_train = training_data.iloc[:, 0]
y_train.head()

In [ ]:
x_train = training_data.iloc[:, 1:]
x_train.head()

Preprocess the test data.

Test data for the hospital dataset is located at `/data/hospital/hospital_test_{party_id}.csv`<br>
Test data for the insurance dataset is located at `/data/insurance/insurance_test_{party_id}.csv`

In [ ]:
# TODO: split the test data into features and labels

Train the model with the training data. Feel free to play with the hyperparameters. For example, you may want to adjust the `n_estimators` hyperparameter, which adjusts the number of trees in the ensemble. For a full list of hyperparameters, go here: https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
# The fit(features, labels) function will train the classifier
arg1 = # TODO: set arg1 to the features
arg2 = # TODO: set arg2 to the labels
model = xgb.XGBClassifier()
model.fit(arg1, arg2)

Get predictions and evaluate the model with the test data. Feel free to use different error functions. We suggest the sklearn `accuracy_score()` function for classification.

In [1]:
# TODO: use the model to get predictions for the test set and calculate the prediction error
preds = model.predict(x_test)
print(accuracy_score(y_test, preds))

Compare your locally trained model's accuracy with those of other members of your federation. How do you think your locally trained model will perform on data that doesn't fit the distribution of your training data?

Once you've finished discussing, let's move to [Exercise 2](Exercise 2.ipynb).